In [ ]:
pip install opencv-python

In [ ]:
import cv2 as cv
import numpy as np


hsv = 0
lower_blue1 = 0
upper_blue1 = 0
lower_blue2 = 0
upper_blue2 = 0
lower_blue3 = 0
upper_blue3 = 0


def nothing(x):
    pass

def mouse_callback(event, x, y, flags, param):
    global hsv, lower_blue1, upper_blue1, lower_blue2, upper_blue2, lower_blue3, upper_blue3, threshold

    # 마우스 왼쪽 버튼 누를시 위치에 있는 픽셀값을 읽어와서 HSV로 변환합니다.
    if event == cv.EVENT_LBUTTONDOWN:
        print(img_color[y, x])
        color = img_color[y, x]

        one_pixel = np.uint8([[color]])
        #cvtColor 함수를 통해 RGB 이미지를 한픽셀로 구성된 이미지를 바꾼 값을 HSV 색공간으로 변환할 수 있다.
        #(uint8 은 0-255 범위를 의미하는 데이터 타입)
        
        hsv = cv.cvtColor(one_pixel, cv.COLOR_BGR2HSV) # color 값은 마우스 이벤트로 선택함에 따라 'color[200, 200, 200]'의 형태로 나올 것이다
        hsv = hsv[0][0] # 리스트 형태에서 [0][0]에 해당하는 픽셀 값을 가져온다.

        threshold = cv.getTrackbarPos('threshold', 'img_result') # threshold 라는 변수를 getTrackbarPos 라는 함수를 통해 현재 트랙바의 위치를 리턴하도록 한다.
        
        # HSV 색공간에서 마우스 클릭으로 얻은 픽셀값과 유사한 필셀값의 범위를 정합니다.
        if hsv[0] < 10:
            print("case1")
            lower_blue1 = np.array([hsv[0]-10+180, threshold, threshold]) # blue1의 최저값을 정하는 것이다. 명도와 채도는 트랙바를 사용하여 변경이 가능하게 지정
            upper_blue1 = np.array([180, 255, 255]) # blue1의 최대값을 정하는 것이다. 명도와 채도는 트랙바를 사용하여 변경이 가능하게 지정
            lower_blue2 = np.array([0, threshold, threshold])
            upper_blue2 = np.array([hsv[0], 255, 255])
            lower_blue3 = np.array([hsv[0], threshold, threshold])
            upper_blue3 = np.array([hsv[0]+10, 255, 255])
            #     print(i-10+180, 180, 0, i)
            #     print(i, i+10)

        elif hsv[0] > 170:
            print("case2")
            lower_blue1 = np.array([hsv[0], threshold, threshold])
            upper_blue1 = np.array([180, 255, 255])
            lower_blue2 = np.array([0, threshold, threshold])
            upper_blue2 = np.array([hsv[0]+10-180, 255, 255])
            lower_blue3 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue3 = np.array([hsv[0], 255, 255])
            #     print(i, 180, 0, i+10-180)
            #     print(i-10, i)
        else:
            print("case3")
            lower_blue1 = np.array([hsv[0], threshold, threshold])
            upper_blue1 = np.array([hsv[0]+10, 255, 255])
            lower_blue2 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue2 = np.array([hsv[0], 255, 255])
            lower_blue3 = np.array([hsv[0]-10, threshold, threshold])
            upper_blue3 = np.array([hsv[0], 255, 255])
            #     print(i, i+10)
            #     print(i-10, i)

        print(hsv[0])
        #print("@1", lower_blue1, "~", upper_blue1)
        #print("@2", lower_blue2, "~", upper_blue2)
        #print("@3", lower_blue3, "~", upper_blue3)


cv.namedWindow('img_color')
cv.setMouseCallback('img_color', mouse_callback)

cv.namedWindow('img_result')
cv.createTrackbar('threshold', 'img_result', 0, 255, nothing) # 트랙바를 원하는 이름('threshold') 으로 원하는 윈도우 창('img_result')에 최소값(0), 최대값(255)을 정하여 만들어 준다. nothing의 경우 트랙바를 사용하기 위해서 def에서 정의한 함수 호출
cv.setTrackbarPos('threshold', 'img_result', 30) # 생성된 트랙바의 초기 설정 값은  30

cap = cv.VideoCapture(0)

while(True):
    #img_color = cv.imread('2.jpg')
    ret,img_color = cap.read()
    height, width = img_color.shape[:2]       
    # shape는 행렬의 차원을 표현하는 방식이고 이미지의 height와 width를 구하기 위해 이용되었으며, [:2]는 이미지의 인덱스 0과 1 을 의미한다. height는 인덱스 0의 크기, width는 인덱스 1의 크기인 것이다.
    
    img_color = cv.resize(img_color, (width, height), interpolation=cv.INTER_AREA)

    # 원본 영상을 HSV 영상으로 변환합니다.
    img_hsv = cv.cvtColor(img_color, cv.COLOR_BGR2HSV)

    # 범위 값으로 HSV 이미지에서 마스크를 생성합니다.
    # lower_blue1, upper_blue1 값등 2, 3번째 파라미터에 각각 이미지 픽셀값의 최저, 최대범위가 입력되고, 첫번째 파라미터로 입력된 것이 그 사이 범위에 포함하는 지를 살펴본다. 포함된다면 흰색, 나머지는 검은색으로 나타난다.
    img_mask1 = cv.inRange(img_hsv, lower_blue1, upper_blue1)
    img_mask2 = cv.inRange(img_hsv, lower_blue2, upper_blue2)
    img_mask3 = cv.inRange(img_hsv, lower_blue3, upper_blue3)
    img_mask = img_mask1 | img_mask2 | img_mask3

    kernel = np.ones((25,25), np.uint8) # 잡음을 줄이기 위한 모폴로지 연산
    # 바이너리 이미지(img_mask)에서 특정픽셀을 0으로 만드는 것이다. () 안의 숫자의 크기가  커질 수록 더 세밀한 제거가 이루어 진다.
    
    img_mask = cv.morphologyEx(img_mask, cv.MORPH_OPEN, kernel) # 노이즈 제거
    img_mask = cv.morphologyEx(img_mask, cv.MORPH_CLOSE, kernel)

    # 바이너리 이미지(img_mask)를 마스크로 사용하여 원본이미지(img_color)에서 범위값(AND연산)에 해당하는 부분을 획득한다.
    img_result = cv.bitwise_and(img_color, img_color, mask=img_mask)


    numOfLabels, img_label, stats, centroids = cv.connectedComponentsWithStats(img_mask)

    for idx, centroid in enumerate(centroids):
        if stats[idx][0] == 0 and stats[idx][1] == 0:
            continue

        if np.any(np.isnan(centroid)):
            continue

        x,y,width,height,area = stats[idx]
        centerX,centerY = int(centroid[0]), int(centroid[1])
        #print(centerX, centerY)
        #print(width, img_result)
        #print(height)

        if area > 50: # 바운딩 박스를 그리는 기준. 이 값을 조명에 따라 조정해준다
            #cv.circle(img_color, (centerX, centerY), 10, (0,0,255), 10)
            cv.rectangle(img_color, (x,y), (x+width,y+height), (0,0,255))
           
        # img_color 창에서 사각형 ROI로 인식 되는 부분의 높이를 추출하는 코드
        cv.putText(img_color, '%d' % (height), (30, 30), cv.FONT_HERSHEY_SCRIPT_COMPLEX, 1, (0, 255, 0)) 

    cv.imshow('img_color', img_color)
    cv.imshow('img_mask', img_mask)
    cv.imshow('img_result', img_result)


    # ESC 키누르면 종료
     if cv.waitKey(1) & 0xFF == 27:
        cap.release()
        cv.destroyAllWindows()
        break